# Code Debugging System

- Author: [HeeWung Song(Dan)](https://github.com/kofsitho87)
- Design: 
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/07-TextSplitter/06-MarkdownHeaderTextSplitter.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/07-TextSplitter/06-MarkdownHeaderTextSplitter.ipynb)

## Overview

In this tutorial, we'll build an AI-powered Python code debugging system using LangGraph. This system automates the debugging process by executing code, analyzing errors, suggesting fixes, and validating corrections.

- **Code Execution**: Run Python code and capture any errors
- **Error Analysis**: Use AI to analyze the error and identify the cause
- **Code Correction**: Generate fixed code and unit tests
- **Validation**: Verify the corrected code works properly

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Basic Components](#basic-components)
- [Building LangGraph Workflow](#building-langgraph-workflow)


### References
- [Debugging complex codebases](https://onsiter.medium.com/debugging-complex-codebases-a-comprehensive-guide-5f8528c48ce4)
----

## Environment Setup

Setting up your environment is the first step. See the [Environment Setup](https://wikidocs.net/257836) guide for more details.

**[Note]**
- The `langchain-opentutorial` is a package of easy-to-use environment setup guidance, useful functions and utilities for tutorials.
- Check out the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_openai",
        "langgraph",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "CodeDebuggingSystem",
    }
)

Alternatively, you can set and load `OPENAI_API_KEY` from a `.env` file.

**[Note]** This is only necessary if you haven't already set `OPENAI_API_KEY` in previous steps.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Basic Components (code lint를 검사하는 부분도 추가해야 함)

Our debugging system is built upon three fundamental utility components that handle code execution, command management, and AI response parsing.

- **Code Executor**: This component provides safe **Python code execution** in an isolated environment, capturing outputs and errors. It serves as the foundation for both initial code testing and validation of corrections.

- **Command Executor**: This component manages system-level operations, particularly package installations. It automatically detects the environment (UV/pip) and ensures proper command execution across different Python environments.

- **Code Block Parser**: The `code_block_parser` is a custom LangChain output parser that processes AI model responses. By extending `BaseOutputParser`, it extracts and categorizes markdown code blocks into structured data, making it seamlessly integrable with LangChain's LCEL pipelines. This parser is crucial for handling AI-generated code corrections and test cases.

These components work together to provide the essential functionality needed by our LangGraph-based debugging workflow.

### Code Executor

First, **Code Executor** is the process of running Python code and capturing any errors that occur during execution.

The `execute_code` function safely executes Python code in a temporary environment and captures comprehensive execution results including output and error messages. It returns a structured result object that serves as the foundation for our debugging workflow's analysis and correction steps.

In [3]:
import subprocess
import sys
import tempfile
import os
from typing import TypedDict, Optional

class ExecutionResult(TypedDict):
    success: bool
    output: Optional[str]
    error: Optional[str]
    code: Optional[str]

def execute_code(input_code, venv_path: str | None=None) -> ExecutionResult:
    """
    파이썬 코드 문자열을 실행하는 함수
    
    Args:
        input_code (str): 실행할 파이썬 코드 문자열
        venv_path (Optional[str]): 가상환경 경로 (선택사항)
    
    Returns:
        ExecutionResult: 실행 결과를 담은 타입이 지정된 딕셔너리
        {
            'success': bool - 실행 성공 여부
            'output': str - 실행 출력 결과
            'error': Optional[str] - 에러 메시지 (있는 경우)
        }
    """
    python_path = sys.executable
    if venv_path:
        python_path = os.path.join(venv_path, "bin", "python")
        if not os.path.exists(python_path):
            raise FileNotFoundError(f"Python executable not found in virtual environment at {venv_path}")

    # 1) 임시 파일에 코드 저장
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as tmp_file:
        tmp_file.write(input_code)
        tmp_file_name = tmp_file.name

    # 2) subprocess로 코드 실행 후 에러 메시지 캡처
    result = subprocess.run(
        [python_path, tmp_file_name],
        capture_output=True, text=True
    )

    # 사용 후 임시 파일 삭제
    os.remove(tmp_file_name)

    # 3) 에러 여부 확인
    if result.returncode != 0:
        error_msg = result.stderr
        # 에러가 발생했으므로 에러 메시지 출력
        # print("[Error] 코드 실행 중 에러 발생:\n", error_msg)

        return ExecutionResult(
            success=False,
            output=result.stdout,
            error=error_msg,
            code=input_code
        )
    else:
        # print("[Success] 코드가 정상적으로 실행되었습니다.")
        # print("Output:", result.stdout)

        return ExecutionResult(
            success=True,
            output=result.stdout,
            error=None,
            code=input_code
        )
    
def execute_code_file(file_path: str, venv_path: Optional[str] = None) -> ExecutionResult:
    """
    파이썬 파일을 실행하는 함수
    
    Args:
        file_path (str): 실행할 파이썬 파일의 경로
        venv_path (Optional[str]): 가상환경 경로 (선택사항)
    
    Returns:
        ExecutionResult: 실행 결과를 담은 타입이 지정된 딕셔너리
        {
            'success': bool - 실행 성공 여부
            'output': str - 실행 출력 결과
            'error': Optional[str] - 에러 메시지 (있는 경우)
        }
    """
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    

    # 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        code = file.read()
    
    # Python 실행 파일 경로 설정
    python_path = sys.executable
    if venv_path:
        python_path = os.path.join(venv_path, "bin", "python")
        if not os.path.exists(python_path):
            raise FileNotFoundError(f"Python executable not found in virtual environment at {venv_path}")

    try:
        # subprocess로 파일 실행
        result = subprocess.run(
            [python_path, file_path],
            capture_output=True,
            text=True
        )

        # 실행 결과 확인
        if result.returncode != 0:
            error_msg = result.stderr
            # print("[Error] 코드 실행 중 에러 발생:\n", error_msg)
            return ExecutionResult(
                success=False,
                output=result.stdout,
                error=error_msg,
                code=code
            )
        else:
            # print("[Success] 코드가 정상적으로 실행되었습니다.")
            # print("Output:", result.stdout)
            return ExecutionResult(
                success=True,
                output=result.stdout,
                error=None,
                code=code
            )

    except Exception as e:
        error_msg = str(e)
        # print(f"[Error] 예외 발생: {error_msg}")
        return ExecutionResult(
            success=False,
            output="",
            error=error_msg,
            code=code
        )

Let's test our debugging system with a Python code sample that contains multiple issues including syntax errors, type errors, and PEP 8 violations. 

This student grade processing code demonstrates common programming mistakes such as missing colons, mixed data types, and naming convention violations that our system will detect and fix.

In [4]:
sample_code = """
# 학생 성적을 시각화하는 코드
import matplotlib.pyplot as plt

# 학생 성적을 처리하는 코드
class student:  # 클래스 이름 컨벤션 위반
    def __init__(self,name,scores):  # 파라미터 간격 컨벤션 위반
        self.name=name  # 연산자 주변 공백 없음
        self.scores=scores
    
    def calculate_average(self):  
        total = 0
        for score in self.scores
            total += score  # 구문 오류: 콜론 누락
        return total/len(self.scores)  # 잠재적 ZeroDivisionError 위험

    def visualize_scores(self):
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(self.scores)), self.scores)
        plt.title(f"{self.name}'s Scores")
        plt.xlabel('Subject')
        plt.ylabel('Score')
        plt.show()
    
    def print_info(self):
        avg = self.calculate_average()
        print(f"Student: {self.name}")
        print(f"Scores: {self.scores}")
        print(f"Average: {avg}")
        self.visualize_scores()


def main():
    # 리스트 내 정수와 문자열 혼합 - 타입 에러 발생 가능
    scores = [85, 90, "95", 88, 92]
    student1 = student("John Doe", scores)
    student1.print_info()

if __name__ == "__main__"
    main()
"""

# test code
result = execute_code(sample_code)

print(f"Success: {result['success']}")
print(f"Error: {result['error']}")
print(f"Output: {result['output']}")

Success: False
Error:   File "/var/folders/8l/h9gthwnn40gcs_zb4k3g4_z40000gn/T/tmpuxxij67t.py", line 13
    for score in self.scores
                            ^
SyntaxError: expected ':'

Output: 


### Command Executor

The `CommandExecutor` provides a unified interface for executing system commands and installing packages across different Python environments. It automatically detects whether to use UV or pip package manager and adapts its behavior accordingly using the Strategy pattern. The system consists of a base executor class with specialized implementations for UV and pip environments, ensuring consistent command execution regardless of the environment setup.

In [5]:
from typing import List, Union
from abc import ABC, abstractmethod

class BaseCommandExecutor(ABC):
    """명령어 실행을 위한 기본 클래스"""
    
    def __init__(self):
        self.python_path = sys.executable
        self.venv_path = os.path.dirname(os.path.dirname(self.python_path))
    
    @abstractmethod
    def install_package(self, package: str) -> bool:
        """패키지 설치 메서드"""
        pass
    
    def run_command(self, command: Union[List[str], str], shell: bool = False) -> bool:
        """일반 명령어 실행"""
        try:
            if isinstance(command, str):
                shell = True
            
            result = subprocess.run(
                command,
                shell=shell,
                text=True,
                capture_output=True,
                check=True
            )
            
            if result.stdout:
                print("Output:", result.stdout)
            if result.stderr:
                print("Stderr:", result.stderr)
            
            return True
            
        except subprocess.CalledProcessError as e:
            print(f"Command failed: {e}")
            print("Error output:", e.stderr)
            return False
        except Exception as e:
            print(f"Unexpected error: {e}")
            return False

class UVCommandExecutor(BaseCommandExecutor):
    """UV를 사용하는 환경을 위한 Executor"""
    
    def install_package(self, package: str) -> bool:
        cmd = ["uv", "pip", "install", package]
        print(f"Installing with UV: {' '.join(cmd)}")
        return self.run_command(cmd)

class VenvCommandExecutor(BaseCommandExecutor):
    """일반 venv 환경을 위한 Executor"""
    
    def install_package(self, package: str) -> bool:
        cmd = [self.python_path, "-m", "pip", "install", package]
        print(f"Installing with pip: {' '.join(cmd)}")
        return self.run_command(cmd)

def get_appropriate_executor() -> BaseCommandExecutor:
    """
    현재 환경에 맞는 Executor 반환
    """
    def check_uv_available() -> bool:
        try:
            result = subprocess.run(
                ["uv", "--version"],
                capture_output=True,
                text=True
            )
            return result.returncode == 0
        except:
            return False
    
    def check_pyproject_toml() -> bool:
        return os.path.exists("pyproject.toml")
    
    # UV 사용 여부 확인
    is_uv = check_uv_available()
    
    if is_uv:
        print("UV environment detected, using UVCommandExecutor")
        return UVCommandExecutor()
    else:
        print("Standard venv environment detected, using VenvCommandExecutor")
        return VenvCommandExecutor()

class CommandExecutor:
    """
    명령어 실행을 위한 통합 인터페이스
    """
    def __init__(self):
        self.executor = get_appropriate_executor()
    
    def execute_bash_commands(self, commands: List[str]) -> bool:
        """
        bash 명령어 목록 실행
        """
        for cmd in commands:
            print(f"\nExecuting: {cmd}")
            
            # pip install 명령어 처리
            if cmd.startswith("pip install"):
                package = cmd.replace("pip install", "").strip()
                success = self.executor.install_package(package)
            else:
                success = self.executor.run_command(cmd, shell=True)
            
            if not success:
                print(f"Failed to execute: {cmd}")
                return False
        return True

In [11]:
# Test CommandExecutor

executor = CommandExecutor()
executor.execute_bash_commands(["pip install matplotlib", "pip install pytest"])

UV environment detected, using UVCommandExecutor

Executing: pip install matplotlib
Installing with UV: uv pip install matplotlib
Stderr: Using Python 3.11.6 environment at: /Users/heewungsong/SideProjects/LangChain-OpenTutorial/.venv
Audited 1 package in 12ms


Executing: pip install pytest
Installing with UV: uv pip install pytest
Stderr: Using Python 3.11.6 environment at: /Users/heewungsong/SideProjects/LangChain-OpenTutorial/.venv
Resolved 4 packages in 51ms
Installed 3 packages in 6ms
 + iniconfig==2.0.0
 + pluggy==1.5.0
 + pytest==8.3.4



True

### Code Block Parser

The `CodeBlockParser` is a custom LangChain output parser that processes AI model responses, extracting and categorizing markdown code blocks into structured data. By extending `BaseOutputParser`, it extracts code blocks with specific tags (**corrected** for fixed code, **tests** for test code) from the AI's response text. This parser enables seamless integration with LangChain's LCEL pipelines while providing a structured way to handle different types of code blocks in our debugging workflow.

In [6]:
import re
from typing import List
from langchain_core.output_parsers import BaseOutputParser
from pydantic import BaseModel, Field

    
class CodeBlocks(BaseModel):
    """언어별 코드 블록을 담는 Dictionary 구조"""
    corrected: List[str] = Field(default_factory=list, description="수정된 코드")
    tests: List[str] = Field(default_factory=list, description="테스트 코드")
    bash: List[str] = Field(default_factory=list, description="Bash 명령어")

class CodeBlockParser(BaseOutputParser[CodeBlocks]):
    """
    마크다운 형식의 코드 블록을 파싱하는 파서
    결과를 언어별로 분류하여 Dictionary 형태로 반환
    """
    
    def parse(self, text: str) -> CodeBlocks:
        """텍스트에서 코드 블록을 추출하여 언어별로 분류"""
        
        # print("Parsing code blocks...")
        # print(text)
        
        # 초기화
        code_blocks = {
            'corrected': [],
            'tests': [],
            'bash': [],
        }
        
        # 수정된 패턴: ```language:type 형식 매칭
        pattern = r"```(\w+):?(\w+)?\n(.*?)\n```"
        matches = re.finditer(pattern, text, re.DOTALL)
        
        for match in matches:
            language = match.group(1).lower()
            block_type = match.group(2) or language  # type이 없으면 language를 type으로 사용
            code = match.group(3).strip()
            
            if block_type in code_blocks:
                code_blocks[block_type].append(code)
        
        return CodeBlocks(**code_blocks)

    @property
    def _type(self) -> str:
        """파서 타입 지정"""
        return "code_block_parser"
    
    def get_format_instructions(self) -> str:
        """파싱 포맷 설명"""
        return """코드는 다음과 같은 마크다운 형식으로 반환되어야 합니다:
        ```language
        your_code_here
        ```
        language는 'python:corrected' 또는 'python:tests' 또는 'bash'여야 합니다.""" 

In [17]:
# Test CodeBlockParser

sample_output_from_llm = """### Error Analysis

1. **Identifying Error Line(s) and Root Cause**:
   - The error occurs at the line `total += score` in the `calculate_average` method. The `scores` list contains a string `"95"` instead of an integer, which leads to a `TypeError` when the code attempts to add this string to an integer (`total`).
   - The root cause is that the `scores` list is not type-checked, allowing incompatible types (strings instead of integers) to be included.

2. **Best Practices to Avoid This Error**:
   - Always validate input data types before performing operations on them.
   - Use type hints in function definitions to clarify expected input types.
   - Consider using exception handling to gracefully manage unexpected data types.

### Code Correction

```python:corrected
class Student:
    def __init__(self, name, scores):
        self.name = name
        # Ensure all scores are converted to integers
        self.scores = [self.validate_score(score) for score in scores]

    def validate_score(self, score):
        try:
            return int(score)  # Convert to int, raises ValueError if conversion fails
        except ValueError:
            raise ValueError(f"Invalid score: {score}. Score must be an integer or convertible to an integer.")

    def calculate_average(self):  
        total = 0
        for score in self.scores:
            total += score
        return total / len(self.scores)

    def print_info(self):
        avg = self.calculate_average()
        print(f"Student: {self.name}")
        print(f"Scores: {self.scores}")
        print(f"Average: {avg}")


def main():
    scores = [85, 90, "95", 88, 92]  # Example input with a string convertible to int
    student1 = Student("John Doe", scores)
    student1.print_info()


if __name__ == "__main__":
    main()
```

### Unit Tests

```python:tests
import unittest

class TestStudent(unittest.TestCase):

    def test_average_normal_case(self):
        student = Student("Jane Doe", [80, 90, 100])
        self.assertEqual(student.calculate_average(), 90.0)

    def test_average_with_string_scores(self):
        student = Student("John Doe", [85, 90, "95", 88, 92])
        self.assertEqual(student.calculate_average(), 90.0)

    def test_invalid_score_type(self):
        with self.assertRaises(ValueError):
            Student("Invalid Student", [85, 90, "Ninety", 88, 92])

    def test_empty_scores_list(self):
        student = Student("Empty Student", [])
        with self.assertRaises(ZeroDivisionError):
            student.calculate_average()

if __name__ == "__main__":
    unittest.main()
```

### Recommendations

1. **Code Quality (PEP 8)**:
   - Ensure class names are capitalized (e.g., `Student` instead of `student`).
   - Use consistent naming conventions and spacing in the code.

2. **Runtime Considerations**:
   - Consider edge cases like empty score lists before performing operations that assume non-empty inputs.

3. **Package Dependencies**:
   - For this simple program, no external packages are required. However, if you expand the functionality (e.g., using a database or web framework), make sure to manage dependencies with a tool like `pip` and maintain a `requirements.txt` file.

By following these recommendations, the code will be more robust, maintainable, and user-friendly.
"""

code_block_parser = CodeBlockParser()
code_block_parser_result = code_block_parser.invoke(sample_output_from_llm)

corrected = "\n".join(code_block_parser_result.corrected)
tests = "\n".join(code_block_parser_result.tests)

print("------- Fixed Code -------")
print(corrected)

print("\n------- Unit Tests -------")
print(tests)

------- Fixed Code -------
class Student:
    def __init__(self, name, scores):
        self.name = name
        # Ensure all scores are converted to integers
        self.scores = [self.validate_score(score) for score in scores]

    def validate_score(self, score):
        try:
            return int(score)  # Convert to int, raises ValueError if conversion fails
        except ValueError:
            raise ValueError(f"Invalid score: {score}. Score must be an integer or convertible to an integer.")

    def calculate_average(self):  
        total = 0
        for score in self.scores:
            total += score
        return total / len(self.scores)

    def print_info(self):
        avg = self.calculate_average()
        print(f"Student: {self.name}")
        print(f"Scores: {self.scores}")
        print(f"Average: {avg}")


def main():
    scores = [85, 90, "95", 88, 92]  # Example input with a string convertible to int
    student1 = Student("John Doe", scores)
    student1.prin

Now, let's try to use the `CodeBlockParser` to parse the output of the LLM.

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template("""You are given a Python code snippet along with an error message that occurs when running the code. Analyze the code, identify the cause of the error, and provide a corrected version of the code.
                                      
---

**Context**:
- Error Message: {ERROR_MESSAGE}
- Original Code: 
{CODE_SNIPPET}
                                      
**Format Requirements**:
- Use **EXACTLY** these code block tags:
  - Corrected code: ```python:corrected
  - Test code: ```python:tests
  - Package installation: ```bash
- Never use generic ```python code blocks

**Response Structure**:

### 1. Error Analysis
- Identify specific line numbers causing the error
- Explain root cause using Python runtime mechanics
- Provide prevention strategies for similar errors

### 2. Code Correction
```python:corrected
# MUST include:
# - Error fixes with line comments
# - Enhanced exception handling
# - PEP 8 compliance
# - Type hints for critical functions
                                      
### 3. Unit Tests
```python:tests              
# REQUIRED in pytest format:
# - Happy path scenario
# - 3+ edge cases
# - Exception validation
# - Mocking for external dependencies     

### 4. Recommendations            
- Package Dependencies(DONT FORGET)
```bash
pip install [detected_packages]                              
```
    - Scan import statements for 3rd-party packages
    - Exclude standard library modules (e.g., os, sys)
    - List packages alphabetically in one-line install command
                                      
- Code quality (PEP 8)
- Runtime considerations
                                      
Format your response with clear sections and tagged code blocks as shown above.
""")

chain = prompt | model | CodeBlockParser()

In [19]:
debug_result = execute_code(sample_code)

response = chain.invoke({
    "ERROR_MESSAGE": debug_result['error'], 
    "CODE_SNIPPET": debug_result['code']
})
print(response)

corrected=['# 학생 성적을 시각화하는 코드\nimport matplotlib.pyplot as plt\nfrom typing import List, Union  # Importing necessary types for type hints\n\n# 학생 성적을 처리하는 코드\nclass Student:  # Changed class name to follow PEP 8 convention\n    def __init__(self, name: str, scores: List[Union[int, str]]):  # Added type hints\n        self.name = name  # Added spacing around \'=\' for PEP 8 compliance\n        self.scores = scores\n    \n    def calculate_average(self) -> float:  # Added return type hint\n        total = 0\n        # Added colon to fix syntax error\n        for score in self.scores:  \n            try:\n                total += int(score)  # Convert score to int, handling potential string inputs\n            except ValueError:\n                print(f"Invalid score \'{score}\' encountered and ignored.")  # Handle non-integer strings\n                continue\n        # Handle potential ZeroDivisionError\n        return total / len(self.scores) if self.scores else 0.0  \n\n    def visua

## Building LangGraph Workflow

In [8]:
from typing import TypedDict, Optional, List, Annotated
from langgraph.graph import StateGraph, END
import operator

# 상태 정의
class AgentState(TypedDict):
    code: str
    error: Optional[str]
    test_results: Optional[str]
    dependencies: List[str]
    execution_result: Optional[ExecutionResult]
    debug_attempts: Annotated[int, operator.add]

# 컴포넌트 초기화
executor = CommandExecutor()
code_parser = CodeBlockParser()
llm_chain = prompt | model

# 노드 정의 함수
def execute_code_node(state: AgentState) -> AgentState:
    """코드 실행 노드"""
    result = execute_code(state["code"])
    return {
        "execution_result": result,
        "error": result["error"] if not result["success"] else None
    }

def analyze_error_node(state: AgentState) -> AgentState:
    """에러 분석 노드"""
    if not state["error"]:
        return state
    
    response = llm_chain.invoke({
        "ERROR_MESSAGE": state["error"],
        "CODE_SNIPPET": state["code"]
    })
    parsed = code_parser.parse(response.content)
    
    return {
        "code": "\n".join(parsed.corrected) if parsed.corrected else state["code"],
        "dependencies": parsed.bash,
        "test_code": "\n".join(parsed.tests) if parsed.tests else None
    }

def install_deps_node(state: AgentState) -> AgentState:
    """의존성 설치 노드"""
    if state["dependencies"]:
        commands = [f"pip install {dep}" for dep in state["dependencies"]]
        executor.execute_bash_commands(commands)
    return state

def run_tests_node(state: AgentState) -> AgentState:
    """테스트 실행 노드"""
    if state.get("test_code"):
        test_result = execute_code(state["test_code"])
        return {"test_results": test_result["output"]}
    return state

def decide_next_step(state: AgentState) -> str:
    """분기 결정 노드"""
    if state["execution_result"]["success"] and not state.get("error"):
        return "end"
    elif state["debug_attempts"] >= 3:
        return "max_attempts_reached"
    return "retry"

# 그래프 구성
builder = StateGraph(AgentState)

# 노드 추가
builder.add_node("execute", execute_code_node)
builder.add_node("analyze", analyze_error_node)
builder.add_node("install", install_deps_node)
builder.add_node("test", run_tests_node)

# 엣지 설정
builder.set_entry_point("execute")
builder.add_edge("execute", "analyze")
builder.add_edge("analyze", "install")
builder.add_edge("install", "test")
builder.add_conditional_edges(
    "test",
    decide_next_step,
    {
        "end": END,
        "retry": "execute",
        "max_attempts_reached": END
    }
)

# 그래프 컴파일
debug_agent = builder.compile()

# 실행 예시
initial_state = {
    "code": sample_code,
    "error": None,
    "dependencies": [],
    "debug_attempts": 0
}

result = debug_agent.invoke(initial_state)

# 결과 출력
print(f"Final Code:\n{result['code']}")
print(f"Test Results:\n{result.get('test_results')}")
print(f"Installed Dependencies: {result['dependencies']}")
print(f"Debug Attempts: {result['debug_attempts']}")

UV environment detected, using UVCommandExecutor

Executing: pip install pip install matplotlib
Installing with UV: uv pip install matplotlib
Stderr: Using Python 3.11.6 environment at: /Users/heewungsong/SideProjects/LangChain-OpenTutorial/.venv
Resolved 11 packages in 33ms
Installed 1 package in 8ms
 + matplotlib==3.10.0


Executing: pip install pip install matplotlib
Installing with UV: uv pip install matplotlib
Stderr: Using Python 3.11.6 environment at: /Users/heewungsong/SideProjects/LangChain-OpenTutorial/.venv
Audited 1 package in 2ms

Final Code:
# 학생 성적을 시각화하는 코드
import matplotlib.pyplot as plt
from typing import List, Union

# 학생 성적을 처리하는 코드
class Student:  # 클래스 이름을 대문자로 시작
    def __init__(self, name: str, scores: List[Union[int, str]]):  # Type hints added
        self.name = name
        self.scores = [int(score) for score in scores]  # Convert scores to integers
    
    def calculate_average(self) -> float:  # Type hint for return type
        total = 0
        for sco